In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
%matplotlib inline

In [14]:
to_numpy = lambda x: x.numpy()
transform = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                    ])
train_dataset = MNIST('.', train=True, download=True, transform=transform)
test_dataset = MNIST('.', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

100.0%
100.0%
100.0%
100.0%


In [ ]:
images_train, labels_train = next(iter(train_loader))

In [16]:
def init_layer(layer, mean=0, std=1):
    # Тут надо быть аккуратным — можно случайно создать копию и менять значения у копии
    weight = layer.state_dict()['weight']
    bias = layer.state_dict()['bias']
    bias.zero_()
    weight.normal_(mean=0, std=std)

def forward_hook(self, input_, output):
    std = input_[0].std().item()
    print('forward', std)

def backward_hook(self, grad_input, grad_output):
    std = grad_input[0].std().item()
    print('backward', std)

In [21]:
layer_1 = nn.Linear(28*28, 500)
layer_2 = nn.Linear(500, 10)

layer_1.register_forward_hook(forward_hook)
layer_2.register_forward_hook(forward_hook)

layer_1.register_backward_hook(backward_hook)
layer_2.register_backward_hook(backward_hook)

std_Xav1 = (6 / (28*28 + 500)) ** 0.5
std_Xav2 = (6 / (500 + 10)) ** 0.5
init_layer(layer_1, -std_Xav1, std_Xav1)
init_layer(layer_2, std_Xav2, std_Xav2)

In [25]:
network = nn.Sequential(
    layer_1,
    nn.Tanh(),
    layer_2
)

n_objects = 100
X = images_train[:n_objects].view(n_objects, -1).data
y = labels_train[:n_objects].data
activations = network(X)
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
loss = loss_fn(activations, y)
loss.backward()

forward 0.9981620907783508
forward 0.7789945006370544
backward 0.027487371116876602
backward 0.015442772768437862


При инициализации весов методом **He** в семинаре получились следующие значения:
forward 0.9945185780525208
forward 0.12716735899448395
backward 0.019999997690320015
backward 0.038504574447870255
В итоге Xavier справился лучше с форвардом чем He, но backward изменения практически неощутимы

In [27]:
class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p

    def forward(self, x):
        mask = torch.empty(X.shape).uniform_(0, 1).type(torch.FloatTensor)
        if self.training:
            mask = torch.where(mask > self.p, 1, 0)
        else:
            mask = torch.where(mask > self.p, 1, 1 - self.p)

        return x * mask

In [29]:
class DropConnect(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DropConnect, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.p = p
    
    def forward(self, x):
        if self.training:
            mask = torch.empty_like(self.linear.weight).bernoulli_(1 - self.p).type(torch.FloatTensor)
            mask = mask / (1 - self.p)
        else:
            mask = torch.ones_like(self.linear.weight) * (1 - self.p)
        mask = mask.requires_grad_(False)
        mask = mask.data
        output = F.linear(x, self.linear.weight * mask, self.linear.bias)
        return output

In [30]:
class TestNetwork(nn.Module):
    def __init__(self, final_part):
        super().__init__()

        channels = 1

        self.conv_layers = nn.Sequential(
            nn.Conv2d(channels, 2, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 4, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )

        #input_size = 7 * 7 * 4 = 196
        self.flatten = nn.Flatten()

        self.final_part = final_part

        self.log_softmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.final_part(x)
        return self.log_softmax(x)

In [ ]:
layers = [
    nn.Linear(196, 150),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(150, 50),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(50, 10)
]

In [32]:
network = TestNetwork(nn.Sequential(*layers))
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
loss_fn = torch.nn.NLLLoss()

In [33]:
for epoch in range(10):
    network.train()
    for batch_idx, (images_train, labels_train) in enumerate(train_loader):
        optimizer.zero_grad()
        X = images_train.data
        y = labels_train.data
        output = network(X)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(X), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

    network.eval()
    losses = []
    for batch_idx, (images_test, labels_test) in enumerate(test_loader):
        X = images_test.data
        y = labels_test.data
        output = network(X)
        loss = loss_fn(output, y)

        losses.append(loss.item())

    print('Test Epoch: {}\tLoss: {:.6f}'.format(
            epoch, np.mean(losses)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.305858
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.894961
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.586218
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.302601
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.384069
Test Epoch: 0	Loss: 0.194472
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.279490
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.373657
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.275959
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.306757
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.364631
Test Epoch: 1	Loss: 0.138301
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.293401
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.208800
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.139512
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.118575
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.215893
Test Epoch: 2	Loss: 0.113630
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.081579
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.156122
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.177839
Train Epoch: 3 [38400/60000 (64%)

In [35]:
layers = [
    nn.Linear(196, 150),
    nn.ReLU(),
    DropConnect(),
    nn.Linear(150, 50),
    nn.ReLU(),
    DropConnect(),
    nn.Linear(50, 10)
]

network = TestNetwork(nn.Sequential(*layers))
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
loss_fn = torch.nn.NLLLoss()

TypeError: DropConnect.__init__() missing 2 required positional arguments: 'input_dim' and 'output_dim'